In [ ]:
# python 3.10.9에서 작성 되었습니다.

### KDT
#### 1. Python 데이터분석: 예측모델 개발

- 데이터 : 밀크티 초등 각 학년별 10000명으로 총 60000명의 데이터


- 분석 결과
    - 전체 60000명 중 이탈자는 총 932명이고 그중 가장 이탈자가 많은 학년은 6학년으로 221명의 이탈자가 있으며 이탈 이유로는 본격적으로 학원을 다니며 중학교 교육 과정의 선행 학습으로 판단된다.

    - 전체 이탈자 932명중 신규 구매자는 639명, 재구매자는 293명으로 확인되며, 1, 2학년 학생들의 경우 이탈자의 약 80 ~ 90% 정도가 신규 구매자이며, 3, 4학년의 경우 약 30%, 5, 6학년의 경우 약 45 ~ 50% 정도로의 비율을 보인다.

    - 전체 학생들의 대상으로 이탈 예측을 했을 경우 기능 중요도(Feature importance)에서 누적 수금액과 누적 유료 학습 일 수가 다른 항목들의 합보다 2, 3배 이상의 중요도를 보이고 있는 것으로 보이고 있다.

    - 이는 이용자 본인이 결제를 해여 사용하는 것이 아니라 학부모들의 결제와 의견이 학생들의 학습일수와 학습시간, 획득점수 보다 큰 영향을 주는 것을 보인다.

    - 따라서 신규 구매자들의 이탈이 가장 많은 1, 2학년 구간에서 이탈자를 최소화 한다면 3, 4, 5, 6학년 까지는 장기적으로 밀크티 초등 서비스를 장기적으로 이용할 것으로 보인다.
    

#### 2. 데이터 불러오기 및 전처리

In [ ]:
# 시작할 때 사용할 패키지
import pandas as pd
import numpy as np
import re
import os

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from statannot import add_stat_annotation

# 그래프 한글 표시 요류 관련 해결 방법
plt.rc('font', family = 'Malgun Gothic')

# 그래프 사이즈 지정
plt.rcParams['figure.figsize'] = [12, 8]

# 그래프 타이틀 한글 출력
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
# 현재 프로세스의 작업 디렉토리 확인
cur_dir = os.getcwd()
print('현재 프로세스의 작업 디렉토리 : ', cur_dir)

# 현재 작업파일의 경로에 있는 리스트 확인 -> 리스트로 반환
dir_list = os.listdir(cur_dir)
print(dir_list)

# 지정된 경로의 디렉토리를 순차적으로 탐색 -> 주로 for문과 활용
for curDir, dirs, files in os.walk(cur_dir):
    print(curDir, dirs, files)
    for f in files :
        print(os.path.join(curDir, f))

In [ ]:
# 데이터 읽어 오기
# 파일명: churn_data.csv
churn_data = pd.read_csv('c:/Users/qkdf1/Documents/Documents/230807/data/churn_data.csv', index_col = 0)
churn_data.head(1)
# 60000 rows × 13 columns

In [ ]:
# 데이터 기본 확인 (전체 레코드 수, 컬럼명 등)
churn_data.info()
# 60000 rows × 13 columns

In [ ]:
# 결측치 확인
null_check_df = churn_data
null_cnt_df = pd.DataFrame(null_check_df.isnull().sum()).rename(columns = {0:'null_count'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count']/len(null_check_df) * 100, 2)
null_cnt_df

In [ ]:
# grade_sect_cd 컬럼 확인
churn_data['grade_sect_cd'].unique()

#### grade_sect_cd을 숫자만 뺴서 grade로 저장

In [ ]:
def change_value(values):
    if values == 'G011':
        return 1
    elif values == 'G012':
        return 2
    elif values == 'G013':
        return 3
    elif values == 'G014':
        return 4
    elif values == 'G015':
        return 5
    elif values == 'G016':
        return 6

In [ ]:
churn_data['grade'] = churn_data['grade_sect_cd'].apply(change_value).astype('int64')
churn_data.head(1)

#### label의 값을 이탈, 미이탈로 변경하여 label_re로 저장

In [ ]:
def change_value(values):
    if values == 0 :
        return '미이탈'
    elif values == 1 :
        return '이탈'

In [ ]:
churn_data['label_re'] = churn_data['label'].apply(change_value)
churn_data.head(1)

#### re_purch의 값을 신규, 재구매로 바꾸어 re_purch_re로 저장

In [ ]:
def change_value(values):
    if values == False :
        return '신규'
    elif values == True :
        return '재구매'

In [ ]:
churn_data['re_purch_re'] = churn_data['re_purch'].apply(change_value)
churn_data.head(1)

#### re_purch의 값을 1, 0으로 변경

In [ ]:
def change_value(values):
    if values == True :
        return 1
    elif values == False :
        return 0

In [ ]:
churn_data['re_purch'] = churn_data['re_purch'].apply(change_value)
churn_data.head(1)

In [ ]:
# 전체 학생들의 신규, 재구매 수
churn_data['re_purch'].value_counts()

In [ ]:
# 이탈자만 따로 빼서 데이터 프레임으로 만들기
churn_data_이탈 = churn_data[churn_data['label_re'] == '이탈']
churn_data_이탈.head(1)

In [ ]:
# 전체 이탈자의 신규, 재구매 수
churn_data_이탈['re_purch_re'].value_counts()

In [ ]:
# 학년별 이탈자 수
churn_data_이탈['grade'].value_counts()

### 학년별 이탈자의 신규, 재구매 수 시각화

In [ ]:
sns.countplot(data = churn_data_이탈, x = 'grade', hue = 're_purch_re', hue_order = ['신규', '재구매'])
plt.xticks(ticks = [0, 1, 2, 3, 4, 5], labels = ['1학년', '2학년', '3학년', '4학년', '5학년', '6학년'])
plt.ylabel('학생 수')
plt.title('학년별 이탈자의 신규, 재구매 수')

### 성별 미이탈 & 이탈

In [ ]:
# 학습 데이터, 레이블 (label) 분리

# 'grade_sect_cd' 값 -> 1, 2, 3, 4, 5, 6 으로 치환 -> 'grade' 컬럼 구성

# 필요 컬럼만 가져오기 (X : 학습 데이터, y: 레이블)
x = churn_data[['grade',
               'tmon_pchrg_lrn_dcnt',
               'acmlt_pchrg_lrn_dcnt',
               'acmlt_bilclct_amt',
               'correct_rate_avg',
               'learning_time_avg',
               'media_action_cnt_sum',
               'non_video_viewed_cnt_sum',
               'get_mm_point_sum',
               're_purch']]

y = churn_data['label']

In [ ]:
# Train, Test 데이터 분리 (이탈 미이탈 값 확인)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

print('[Trian] 전체 :{}, 이탈 : {}, 미이탈 {}'.format(len(X_train), y_train[y_train == 1].shape[0], y_train[y_train == 0].shape[0]))
print('[test] 전체 :{}, 이탈 : {}, 미이탈 {}'.format(len(X_test), y_test[y_test == 1].shape[0], y_test[y_test == 0].shape[0]))

In [ ]:
# LGBM 모델 학습
# pip install lightgbm
from lightgbm import LGBMClassifier
LGBMC_model = LGBMClassifier(random_state = 0, verbose = -1)
LGBMC_model.fit(X_train, y_train)

In [ ]:
# 예측 결과
# 예측 및 결과
y_pred = LGBMC_model.predict(X_test)
y_pred = list(y_pred)
print('[Predict] 이탈 : {}, 미이탈 : {}'.format(y_pred.count(1), y_pred.count(0)))
# y_test : 결과 값 (실제 값)

In [ ]:
# Accuracy 측정
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
# precision, recall, f1 score 측정
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, y_pred, average = 'binary')

In [ ]:
# 영향도 (주요 변수 시각화)
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize = (5, 5))
plot_importance(LGBMC_model, ax = ax)

In [ ]:
def churn_model(grade_df, grade_name = '1') :
    
    x = grade_df[['tmon_pchrg_lrn_dcnt',
                  'acmlt_pchrg_lrn_dcnt',
                  'acmlt_bilclct_amt',
                  'correct_rate_avg',
                  'learning_time_avg',
                  'media_action_cnt_sum',
                  'non_video_viewed_cnt_sum',
                  'get_mm_point_sum',
                  're_purch']]
    
    y = grade_df['label']
    
    # Train, Test 데이터 분리 (이탈 미이탈 값 확인)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
    print('[Trian] {}학년, 전체 : {}, 이탈 : {}, 미이탈 {}'.format(grade_name, len(X_train), y_train[y_train == 1].shape[0], y_train[y_train == 0].shape[0]))
    print('[test] {}학년, 전체 : {}, 이탈 : {}, 미이탈 {}'.format(grade_name, len(X_test), y_test[y_test == 1].shape[0], y_test[y_test == 0].shape[0]))

    # LGBM 모델 학습
    LGBMC_model = LGBMClassifier(random_state = 0, verbose = -1)
    LGBMC_model.fit(X_train, y_train)

    # 예측 및 결과
    y_pred = LGBMC_model.predict(X_test)
    y_pred = list(y_pred)
    print('[Predict] {}학년, 이탈 : {}, 미이탈 {}'.format(grade_name, y_pred.count(1), y_pred.count(0)))

    # Accuracy 측정
    acc = accuracy_score(y_test, y_pred)
    
    # precision, recall, f1 score 측정
    pre, re, f1, _ = precision_recall_fscore_support(y_test, y_pred, average = 'binary')
    print('{}학년, acc : {}, precision : {}, recall : {}, f1 : {}'.format(grade_name, acc, pre, re, f1))
    
    # 영향도 (주요 변수 시각화)
    fig, ax = plt.subplots(figsize = (5, 5))
    plot_importance(LGBMC_model, ax = ax)
    plt.title('{}학년 Feature Importance'.format(grade_name))
    plt.show()
    return

In [ ]:
# 실행부 구성
for grade in range(1, 7) :
    grade_df = churn_data[churn_data['grade'] == grade]
    churn_model(grade_df, grade_name = str(grade))